In [1]:
import pandas as pd
import numpy as np

In [15]:
df_p = pd.read_csv("data/processed_data.csv")

In [16]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df_p, test_size=0.2)
train_df, val_df = train_test_split(train_df, test_size=0.2)

In [25]:
#Train set
x_train = torch.from_numpy(train_df.drop(columns = 'Target').values).unsqueeze(1)
y_train = torch.from_numpy(train_df['Target'].values)
#Validation set
x_val = torch.from_numpy(val_df.drop(columns = 'Target').values).unsqueeze(1)
y_val = torch.from_numpy(val_df['Target'].values)
# Test set
x_test =torch.from_numpy( test_df.drop(columns = 'Target').values).unsqueeze(1)
y_test =torch.from_numpy( test_df['Target'].values)

In [10]:
from pylab import *
assert len(np.where (isnan((tr_labs)) == True)[0]) == 0


In [14]:
from fastai.data.core import DataLoaders
from fastai.vision.all import *
import torch.nn as nn
import torch.nn.functional as F 

In [96]:
class Shallow(nn.Module):
    """
    VGG 4 layers
    """
    def __init__(self):
        super(Shallow, self).__init__()
        power = 4

        self.conv1 = nn.Conv1d(1, 2**power, 4, 2)
        self.bn1 = nn.BatchNorm1d(2**power)
        #self.pool1 = nn.MaxPool1d(2)

        self.conv2 = nn.Conv1d(2**power, 2**power, 4, stride = 2)
        self.bn2 = nn.BatchNorm1d(2**power)
        #self.pool2 = nn.MaxPool1d(1)

        self.conv3 = nn.Conv1d(2**power, 2**(power+1), 2, stride = 2)
        self.bn3 = nn.BatchNorm1d(2**(power+1))
        self.pool3 = nn.MaxPool1d(2)

        self.flatten = nn.Flatten()
        self.fc = nn.Linear(32, 2)
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x).relu()
        #x = self.pool1(x)

        x = self.conv2(x)
        x = self.bn2(x).relu()
        #x = self.pool2(x)
        x = self.conv3(x)
        x = self.bn3(x).relu()
        x = self.pool3(x)
        x = self.fc(self.flatten(x))
        
        return x

In [92]:
trainset = Dataset(train_df)
sampler = WSampler(trainset, batch_size = 10,weights = [0.5]*2 )
train_loader = torch.utils.data.DataLoader(trainset, batch_size=10, 
                                     num_workers=0, sampler = sampler)
x_batch, y_batch = next(iter(train_loader))

In [93]:
#model
val_generator = Dataset(val_df)
val_sampler =  WSampler(val_generator, batch_size = 30, weights = [0.5]*2)
val_dataset = torch.utils.data.Subset(val_generator, indices= list(val_sampler))
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=30, 
                                     num_workers=0)


In [98]:
model = Shallow()
model.float()
#model = model.to(torch.float)
# loss_fn 
loss_fn = CrossEntropyLossFlat() #soft_margin_loss
# fastai learner
dls = DataLoaders(train_loader,val_loader) # a validation set need to be defined
learn = Learner(dls, model, loss_func=loss_fn, opt_func=Adam, metrics=[accuracy, F1Score(), Precision(), Recall()], 
                )

In [99]:
learn.fit(1)

In [100]:
learn.validate()